functions

In [ ]:
from re import T
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import pandas as pd
def compute_full_extracted_triples(intput_sentence,keywords=None,hightlight=None,enhance=False):
    #v2.1
    extra_prompt=""
    extra_prompt_pair=""
    extra_prompt_hightlight=""
    if keywords!=None and len(keywords)>0 and enhance == True:
        extra_prompt = "When you find those keywords in above sentence, you should pay more attention to them and extract more triples about them."+str(keywords)
        #if len(keywords)!=1:
            #pairs = [f"'{keywords[i]}' AND '{keywords[j]}'" for i in range(len(keywords)) for j in range(i+1, len(keywords))]
            #extra_prompt_pair ="Rule 12, these following two subjects/objects may have hidden relationships"+str(pairs)+"You may create new triples based on the hidden relationships. For example, given triple [Formbook, download, plugin] and [plugin, is, XLoader], you should extract [Formbook, download, XLoader]."

    if hightlight!=None and len(hightlight)>0 and enhance == True:
        extra_prompt_hightlight="Those sentences contain some important information, you should pay more attention to them and extract more triples about them."+str(hightlight)
    
    def generate_prompt_postprocess(text):
        promptmessage = [
        {
        "role": "user",
        "content": 
        ''' 
        You play the role of an entity extraction expert and modify/simplify/split the text (extracted multiple triples) in the entity extraction result I gave you (a python dictionary with key as the source sentence with ellipsis and value as the extracted triples) according to the following rules. A triple is a basic data structure used to represent knowledge graphs, which are structured semantic knowledge bases that describe concepts and their relationships in the physical world. A triple consists of three elements: [SUBJECT, RELATION,OBJECT]. The subject and the object are entities, which can be things, people, places, events, or abstract concepts. The relation is a relation that connects the subject and the object, and expresses how they are related. For example, [Formbook, is, malware] is a triple that describes the relationship between the malware Formbook and the concept of malware.
        Rule 1: If the subject or object in a triple contains pronouns such as it, they, malware, Trojan, attack, ransomware, or group, replace them with a specific name as much as possible according to the context, such as “CVE-xxx” or “XLoader” will replace "it" or "malware" if context has this relationship information.
        Rule 2: Focus on malware, Trojan horse, CVE, or hacking organization as the subject of the triples, if a subject with "malware" or "Trojan horse" or "CVE" or "hacking organization" is found and has additional suffixes, remove the suffixes.
        Rule 3: Split a complex triple into multiple simpler forms. For example, [Formbook and XLoader, are,malware] should be split into [Formbook,is,malware] and [XLoader,is,malware].
        Rule 4: If the [subject,relation] in a triple can be formed into a new [subject,relation,object] triple because relation itself has a new object in it, create a new triple while keeping the original one. 
        Rule 5: If the object can be simplified to a more concise, generic expression, create a new triple while keeping the original one. For example, ["Formbook", "save", "XLoader in desktop"] MUST has a new triple ["Formbook", "save", "XLoader"] due to the object "XLoader in desktop" can be simplified to "XLoader".
        Rule 6: Simplify the subject, object, and relation into a more concise, generic expression.
        Rule 7:When you encounter a subject or object that contains modifiers and adjectives, remove them. For example, [a notorious Formbook malware] should be simplified to [Formbook].
        Rule 8:When you encounter a plural or past tense form, convert it to singular or present tense. For example, [Windows users] should be converted to [Windows user].
        Rule 9:When you encounter an MD5, registry, path, or other identifier that contains prefixes, remove them. For example, [md5 xxxxx] should be simplified to [xxxxx].
        Rule 10:When you encounter a proper noun that contains a suffix, remove the suffix. For example, [“Specific names of a malware/ransomware/trojan” malware/ransomware/trojan] should be simplified to [“Specific names of a malware/ransomware/trojan”]
        Rule 11: Make sure the subject has a prefix "SUBJECT:", the relation has prefix "RELATION:", the object has prefix "OBJECT:", a triple example is "[SUBJECT:Formbook, RELATION:save, OBJECT:a file]
        '''
        +extra_prompt_pair+"Here is my entity extraction result:\""+str(text)+"\".Now, you apply the rules I told you before. Write down your though, think it step by step. If all triple don't need to be modified based on specific rule, just rewrite all triples that don't need to be modified again. In the end, you MUST tell me the final new entity extraction result. Make sure your results start with \"[[\" and end with \"]]\"."
        },
        ]
        return promptmessage
    
    def generate_prompt(text):
            promptmessage = [
            {
            "role": "user",
            "content": 
            ''' 
            As an AI trained in entity extraction and relationship extraction. You're an advanced AI expert, so even if I give you a complex sentence, you'll still be able to perform the relationship extraction task. The ouput of the task is a list of list.
            A triple is a basic data structure used to represent knowledge graphs, which are structured semantic knowledge bases that describe concepts and their relationships in the physical world. A triple MUST has THREE elements: [Subject, Relation,  Object]. For example, "[Subject:FinSpy malware, Relation:was the final payload]"(2 elements) and "[Subject:FinSpy malware, Relation:was, Object:the final payload, None:that will be used]"(4 elements) do not contain exactly 3 elements and should not exist in your result. The subject and the object should be Noun. The relation should be a relation words that connects the subject and the object, and expresses how they are related. 
            
            You only extracts triples that are related to computer science, that means focus on those sentences that explicitly involve concepts and entities related to the field of computer science. For instance, sentences mentioning programming languages, software and hardware technologies, algorithms, network protocols, security vulnerabilities, operating systems, etc., are all related to computer science. Examples of valid triples include [Formbook, is, malware] because it describes relationships between entities within the field of computer science.

            Conversely, triples should not be extracted from content unrelated to computer science. For example, [The Sun, is, a star], although a valid relational description, should be excluded because it pertains to astronomy, not computer science. Similarly, [Picasso, is, a painter] should also not be considered, as it involves the field of art, not computer science.
            
            If my input sentences do not have any triple about computer science,you must output the answer as [[Blank placeholders since the current text does not contain any computer science related triples]]. If you find that you are keeping output triples that are not related to computer science, you should stop as soon as possible and output the result as [[Blank placeholders since the current text does not contain any computer science related triples]]\
            '''
            },
            {
            "role": "assistant",
            "content": "I got it."
            },
            {
            "role": "user",
            "content": "Here is one sentence from example article:\"Leafminer attempts to infiltrate target networks through various means of intrusion: watering hole websites, vulnerability scans of network services on the internet, and brute-force/dictionary login attempts.\""
            },
            {
            "role": "assistant",
            "content": "[[SUBJECT:Leafminer,RELATION:attempts to infiltrate,OBJECT:target networks],[SUBJECT:Leafminer,RELATION:use,OBJECT:watering hole websites],[SUBJECT:Leafminer,RELATION:use,OBJECT:vulnerability scans of network services on the internet],[SUBJECT:Leafminer,RELATION:use,OBJECT:brute-force],[SUBJECT:Leafminer,RELATION:use,OBJECT:dictionary login attempts]]."
            },
            {
            "role": "user",
            "content": "Here is one sentence from example article:\"Kismet is also a powerful tool for penetration testers that need to better understand their target and perform wireless LAN discovery.\""
            },
            {
            "role": "assistant",
            "content": "[[SUBJECT:Kismet,RELATION:is a powerful tool for, OBJECT:penetration testers],[SUBJECT:testers, RELATION:understand, OBJECT:their target],[SUBJECT:testers,RELATION: perform, OBJECT:wireless LAN discovery]]"
            },
            {
            "role": "user",
            "content": "Here is one sentence from example article:\"Legendary Pokémon , or Pokémon Illusions are extremely rare and often very powerful Pokémon that are often associated with legends of creation and/or destruction within their endemic regions. \""
            },
            {
            "role": "assistant",
            "content": "This text does not contain any computer science related triples, my output is [[Blank placeholders since the current text does not contain any computer science related triples]]"
            },
            {
            "role": "user",
            "content": "Here is one sentence from example article:\"The Royal Knights  are a group of thirteen Mega-level[1] Holy Warrior Digimon[2] that are the Digital World's sacred guardians,[3] and are famed among Digimon as guardian deities of the Computer Network.[4][5] The group was founded by Imperialdramon Paladin Mode,"
            },
            {
            "role": "assistant",
            "content": "[[The Royal Knights, are, a group of thirteen Mega-level Holy Warrior Digimon],[The Royal Knights, are, the Digital World's sacred guardians],[The Royal Knights, are, guardian deities of the Computer Network],[The group, was founded by, Imperialdramon Paladin Mode]]. I find that the sentence does not contain any computer science related triples, and I am keeping output triples that are not related to computer science. So I stop here and output the result as [[Blank placeholders since the current text does not contain any computer science related triples]]."
            },
            {"role": "user",
            "content": 
            """
            Here are my new sentence, extract all possible entity triples from it. Now, I start to give you sentence.\""
            """+text+
            """\"Now, my input text are over. You MUST follow the rules I told you before. 
            """+extra_prompt+extra_prompt_hightlight
            },
            ]
            return promptmessage

    def generate_prompt_basedon3(inSent,inlist):
        promptmessage = [
        {
        "role": "user",
        "content":"Your primary objective is to consolidate entity extraction results derived from sentence or sentences by difference distinct assistants into a singular, coherent output. This process necessitates a careful analysis to identify and merge triples that, despite potentially differing in phrasing, communicate identical information. Additionally, it is imperative to filter out any triples that fail to meet the specified criteria for validity and relevance.Key Points: Eliminate Invalid Triples:Any triples not conforming to the structure of having exactly one SUBJECT, one RELATION, and one OBJECT should be removed.Exclude triples missing any of the three essential components [SUBJECT, RELATION, OBJECT] .Discard any triples that introduce information not explicitly mentioned in the provided sentence.Consolidation means maperge triples that convey the same information but are expressed using different terminologies or structures. This involves combining synonyms or semantically similar phrases that refer to the same entities or actions.Examples for Reference: Consider the sentence FinSpy is a surveillance software designed by Gamma Group to spy on digital communications. Analysis of extraction results from different assistants might yield the following triples: Assistant A: [FinSpy, designed by, Gamma Group], [FinSpy, to spy on, digital communications] Assistant B: [FinSpy, is designed by, Gamma Group], [Surveillance software, is used for, spying on digital communications] Assistant C: [FinSpy, developed by, Gamma Group], [FinSpy, monitors, communications] Unified and Consolidated Output: [FinSpy, designed by, Gamma Group] [FinSpy, to spy on, digital communications] This output exemplifies the elimination of redundant or irrelevant triples and the merging of equivalent triples into a single, consolidated form. Now, the source sentences for extracted triples tasks are'+"+str(inSent)+', the extracted triples result are'+str(inlist)+"Just answer me your final answer without any other words, and only start with \"[[\", and only end with \"]]\"."
        },
        ]
        return promptmessage
        
    def clean_text(text):
        import string,re
        if not isinstance(text, str):
            return text
        cleaned_text = re.sub(r'[^\x20-\x7E]', '', text)
        cleaned_text = re.sub(r'[\s{}]+'.format(re.escape(string.punctuation)), '', cleaned_text)
        cleaned_text = re.sub(r'SUBJECT|RELATION|OBJECT', '', cleaned_text)
        return cleaned_text if cleaned_text else 'Null'

    def get_only_triples(text):
        import re
        text = text.replace('"', '').replace("'", "")
        text = text.replace('\n', '')
        text = re.sub(r'\[\s+', '[', text)
        text = re.sub(r'\s+\]', ']', text)
        text = re.sub(r',\s+', ',', text)
        text = re.sub(r'\[\s+', '[', text)
        if '[[' in text and ']]' in text:
            return text[text.rindex('[['):text.rindex(']]')+2]
        return text

    single_sentence=intput_sentence
    
    content_first_extraction=''
    first_answer_list=[]
    temperature_list=[0.2]*3
    from openai import OpenAI
    api_key = "EMPTY"
    api_base = "http://localhost:8000/v1"
    client = OpenAI(api_key=api_key, base_url=api_base)
    import concurrent.futures
    print(generate_prompt(single_sentence))
    def process_temperature(temperature):
        stream = client.chat.completions.create(
            model='./cti-qwen1.5-70b-awq',
            messages=generate_prompt(single_sentence),
            stream=True,
            max_tokens=6000,
            temperature=temperature,
            extra_body={
            "stop_token_ids": [7]
            }
        )
        final_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                final_response += chunk.choices[0].delta.content
        content_first_extraction = final_response
        cleaned_text = clean_text(str(content_first_extraction))
        if any(keyword in cleaned_text for keyword in ['CVExxx', 'Formbook', 'XLoader', 'Malwaresavetextfile', 'Leafminer', 'FinSpy', 'Kismet', 'Specificnamesofa']):
            return 'ERROR'
        else:
            return get_only_triples(content_first_extraction)

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        first_answer_list = list(executor.map(process_temperature, temperature_list))
    
    def check_placeholders_in_list(sublist, placeholder="Blank placeholders"):

        count = sum(1 for item in sublist[:3] if placeholder in item)
                return count >= 2
    
    blank_flag=check_placeholders_in_list(first_answer_list)
  
    if blank_flag :
        extracted_text='This text does not contain any computer science related triples.'
    else:
        stream = client.chat.completions.create(
                model='./cti-qwen1.5-70b-awq',
                messages=generate_prompt_basedon3(single_sentence,first_answer_list[0:3]),
                stream=True,
                max_tokens=8000,
                temperature=0.5,
                extra_body={
                "stop_token_ids": [7]
                }
            )

        final_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                final_response += chunk.choices[0].delta.content  
        content_first_extraction_merged = final_response
        content_first_extraction_merged=get_only_triples(content_first_extraction_merged)
        stream = client.chat.completions.create(
                model='./cti-qwen1.5-70b-awq',
                messages=generate_prompt_postprocess(content_first_extraction_merged),
                stream=True,
                max_tokens=10000,
                temperature=0.7,
                extra_body={
                "stop_token_ids": [7]
                }
            )
        final_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                #print(chunk.choices[0].delta.content, end="")
                final_response += chunk.choices[0].delta.content
        content_simple_version=final_response  
        extracted_text = get_only_triples(content_simple_version)
    
    return extracted_text

def clean_full_extracted_triples(text):
    #remvoe all \n
    text=text.replace('\n','')
    #remove the space between ‘:" and "["
    import re
    text=re.sub(r':\s+\[',r'[',text)
    #remove the space between ‘[" and "["
    text=re.sub(r'\s+\[',r'[',text)
    #remove the space between ‘]" and "]"
    text=re.sub(r'\s+\]',r']',text)
    #replace ], ] or ],] or ] ,] with ]]
    text = re.sub(r'\]\s*,\s*\]', ']]', text)

    triple_only_text=text
    #if [[ and ]] in text, extract the content between them with a [ and ]
    if "[[" in text and "]]" in text:
        start_index = text.rindex('[[')+1
        end_index = text.rindex(']]') + 1
        triple_only_text = text[start_index:end_index]
    else:
        if "[["in text or "]]" in text:
            start_index = text.index('[')
            end_index = text.rindex(']') + 1
            triple_only_text = text[start_index:end_index]
    #remove all " and ' in text
    triple_only_text=triple_only_text.replace('"','')
    triple_only_text=triple_only_text.replace("'",'')

    return triple_only_text

def merge_extracted_triples(longmem,shortmem,sentence):
    #v1.5
    def generate_prompt(longmem,shortmem,sentence):
        promptmessage = [
        {
        "role": "user",
        "content": 
        '''You are a triples integration assistant. Triple is a basic data structure, which describes concepts and their relationships. A triple in long-term and short-term memory MUST has THREE elements: [Subject, Relation, Object]. You are now reading a whole article and extract all triples from it. But you can only see part of the article at a time. In order to record all the triples from a article, you have the following long-term memory area to record the triples from the entire article. long-term memory stores information on the aricle parts you have already read.
        -The start of the long-term memory area-
        #Triples will be added here
        -The end of the short-term memory area-
        Second, you now see a part of this article. Based on this part, you already extract such triples and place them in your short-term memory: 
        -The start of the short-term memory area-
        #Triples will be added here
       -The end of the short-term memory area-
        Third, now review your long-term memory and short-term memory. Modify the short-term memory into a new short-term memory. You should follow following rules to modify triples in short-term memory to make them consistent with triples in long-term memory. You should write down how you use the rule to modify the triples in short-term memory. 
        
        Rule 1. You notice that in these triples, some triples have subjects and objects that contain partially identical terms and refer to the same specific nouns, but these specific nouns have prefixes/suffixes/modifiers that make them not identical. You should delete the prefixes/suffixes/modifiers and unify them into the same specific nouns.
        
        Before rule: [the Formbook, is designed to run as, a deleter] [Formbook sample, is designed to run as, one-time encryptor]

        After rule: [Formbook, is designed to run as, a deleter] [Formbook, is designed to run as, one-time encryptor]

        Explanation: The words "the Formbook" and "Formbook sample" refer to the same entity, so they are unified to use the exact same subject "Formbook" for consistency.
        
        Rule 2. Be especially careful that when you meet specific names of malware,CVE, Trojans, hacker organizations, etc., always use their specific names and remove the prefixes/suffixes/modifiers.
        
        Before rule: [Malware Formbook, is, malware] 
        
        After rule: [Formbook, is, malware]
        
        Explanation: The word "Formbook" is a specific name of malware, so it should be used as the subject of the triple and the prefix "Malware" should be removed.
        
        Rule 3. Don't add unexisting triples to your new short-term memory. 
    
        Suppose you find in long-term memory: [the malware, download, Leafminer] and in short-term memory: [Formbook, is, malware]. You cannot add a new triple in new short term memory: [Formbook, download, Leafminer]. Because you don't have evidence that "the malware" in the long-term memory specifically refers to "Formbook".
        
        Rule 4. Don't add unexisting triples that don't exsit in long-term memory or short-term memory to your new short-term memory. You should add triples from long-term memory or short-term memory to your new short-term memory, not from your imagination and selfcreation
        
        Rule 5. Don't add any example word like 'Formbook','XLoader','Leafminer', 'FinSpy', 'Kismet' in your new short-term memory area, they are just example words not the real triples in the long term memory area or short term memory area.
        
        Rule 6. new short-term memory area must be started with \'-The start of new short-term memory area-\' and ended with \'-The end of new short-term memory area-\'. A triple in new short-term memory MUST has THREE elements: [Subject, Relation, Object]. 
        '''
        },
        {"role": "assistant",'content': 'Yes, I understand and totally follow the rules.'},
        {"role": "user",'content': '''
         -The start of the long-term memory area-  
         [SUBJECT:AgumonHacker, RELATION:disrupts, OBJECT:network operations], [SUBJECT:HackGroup Gabumon, RELATION:targets, OBJECT:financial institutions], [SUBJECT:Malware Biyomon, RELATION:steals, OBJECT:personal data], [SUBJECT:CyberAttack 2042, RELATION:compromised, OBJECT:database systems],  [SUBJECT:A sample of Biyomon, RELATION:save, OBJECT:a payload]
         -The end of the long-term memory area-

        -The start of the short-term memory area-  
         [SUBJECT:The AgumonHacker Trojan, RELATION:created by, OBJECT:a sophisticated hacker group], [SUBJECT:Gabumon hackers, RELATION:launched, OBJECT:series of phishing campaigns], [SUBJECT:Biyomon malware program, RELATION:encrypts, OBJECT:files for ransom], [SUBJECT:Attack in 2042, RELATION:led to, OBJECT:sensitive leaks],[SUBJECT:Biyomon, RELATION:run, OBJECT:payload]
        -The end of the short-term memory area-
         '''
         },
        {"role": "assistant",'content': 
         '''
         Modifications required based on the rules:
        Apply Rule 1 and Rule 2: Unify subjects and objects with partially identical terms and remove any prefixes/suffixes/modifiers, particularly for specific names like malware and hacker groups.
        Modified short-term memory:

        [SUBJECT:The AgumonHacker Trojan, RELATION:created by, OBJECT:a sophisticated hacker group] → [SUBJECT:AgumonHacker, RELATION:created by, OBJECT:sophisticated hacker group]
        [SUBJECT:Gabumon hackers, RELATION:launched, OBJECT:series of phishing campaigns] → [SUBJECT:HackGroup Gabumon, RELATION:launched, OBJECT:phishing campaigns]
        [SUBJECT:Biyomon malware program, RELATION:encrypts, OBJECT:files for ransom] → [SUBJECT:Malware Biyomon, RELATION:encrypts, OBJECT:files for ransom]
        [SUBJECT:Attack in 2042, RELATION:led to, OBJECT:sensitive leaks] → [SUBJECT:CyberAttack 2042, RELATION:led to, OBJECT:sensitive data leaks]

        I also find a triple in long-term memory that needs to be modified based on Rule 1 and Rule 2:

        [SUBJECT:A sample of Biyomon, RELATION:save, OBJECT:a payload] → [SUBJECT:Biyomon, RELATION:save, OBJECT:payload]

        Apply Rule 3 and 4 and 5: We must ensure we do not create unexisting triples when merging the memories.
        After applying the modifications, we now create a new short-term memory area:

        -The start of new short-term memory area- 
        [SUBJECT:AgumonHacker, RELATION:created by, OBJECT:sophisticated hacker group], 
        [SUBJECT:HackGroup Gabumon, RELATION:launched, OBJECT:phishing campaigns],
        [SUBJECT:Biyomon, RELATION:encrypts, OBJECT:files for ransom],
        [SUBJECT:CyberAttack 2042, RELATION:led to, OBJECT:sensitive data leaks],
        [SUBJECT:Biyomon, RELATION:run, OBJECT:payload],
        [SUBJECT:Biyomon, RELATION:save, OBJECT:payload] 
        -The end of new short-term memory area-
         '''   },
        {"role": "user",'content': 
        '''
        Good. Now, let's swtich to another article. 
        -The start of the long-term memory area-
        '''+str(longmem)+'''
        -The end of the long-term memory area-
    
        -The start of the short-term memory area-
        '''+str(shortmem)+'''
        -The end of the short-term memory area-
        
        Now, follow the rules. Write down how you use the rule to modify the triples in short-term memory. If there is no any triple in my input short-term memory, you still need to write down \'-The start of new short-term memory area-\' and ended with \'-The end of new short-term memory area-\'. with only one blank line between them. If my input short-term memory is already perfect, you still need to write down \'-The start of new short-term memory area-\' and content of that perfect short-term memory and ended with \'-The end of new short-term memory area-\'.
        '''
        },      
        ]
        return promptmessage

    import os,openai
    openai.api_base = "http://localhost:8080/v1"
    openai.api_key = ""
    model = "Empty"
    import concurrent.futures
    
    def clean_text(text):
        import string,re
        if not isinstance(text, str):
            return text
        cleaned_text = re.sub(r'[^\x20-\x7E]', '', text)
        cleaned_text = re.sub(r'[\s{}]+'.format(re.escape(string.punctuation)), '', cleaned_text)
        cleaned_text = re.sub(r'SUBJECT|RELATION|OBJECT', '', cleaned_text)
        return cleaned_text if cleaned_text else 'Null'
    import re
    from tqdm import tqdm
    retry_times=0
    pass_flag=False
    from openai import OpenAI
    
    api_key = "EMPTY"
    api_base = "http://localhost:8000/v1"

    client = OpenAI(api_key=api_key, base_url=api_base)
    
    stream = client.chat.completions.create(
        model="./cti-qwen1.5-70b-awq",
        messages=generate_prompt(longmem,shortmem,sentence),
        stream=True,
        max_tokens=10000,
        temperature=0.7,
        extra_body={
        "stop_token_ids": [7]
        }
    )
    final_response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            final_response += chunk.choices[0].delta.content  
    fullanswer=final_response
   
    return fullanswer   

def check_brackets(my_string):
    if my_string is None or len(my_string) == 0:
        return False

    my_string = my_string.strip()

    first_ten_chars = my_string[:10]
    last_ten_chars = my_string[-10:]
    
    found_bracket_in_first_ten = '[' in first_ten_chars

    found_bracket_in_last_ten = ']' in last_ten_chars

    return found_bracket_in_first_ten and found_bracket_in_last_ten

def full_text_to_parts(text):
    import nltk
    paragraphs = text.split('\n')
    
    processed_paragraphs = []

    for paragraph in paragraphs:
        if len(paragraph) > 600:
            sentences = nltk.sent_tokenize(paragraph)
            sentences = [x[0:500] for x in sentences]
            new_paragraph = ''
            for sentence in sentences:
                temp_length = len(new_paragraph) + len(sentence)
                if temp_length < 600:
                    new_paragraph += (sentence + '\n')
                else:
                    if len(new_paragraph) >= 20:
                        processed_paragraphs.append(new_paragraph.strip())
                    new_paragraph = sentence + '\n'
            if len(new_paragraph) >= 20:
                processed_paragraphs.append(new_paragraph.strip())
        else:
            if len(paragraph) >= 20:
                processed_paragraphs.append(paragraph.strip())

    combined_paragraphs = []
    current_combined = ''

    for paragraph in processed_paragraphs:
        temp_length = len(current_combined) + len(paragraph) + 1 
        if temp_length < 600:
            current_combined += (' ' if current_combined else '') + paragraph
        else:
            combined_paragraphs.append(current_combined)
            current_combined = paragraph
            
    if current_combined:
        combined_paragraphs.append(current_combined)

    return combined_paragraphs

def name_entity_recognition(text):
  def ask(prompt, token, temp, model,streamprint=True):
      import os
      from openai import OpenAI
      if model == "gpt4":
          os.environ["OPENAI_API_KEY"] = ""
          api_key = os.getenv("OPENAI_API_KEY")
          api_base = 'https://api.openai.com/v1'
          setmodel = ''
      else:
          setmodel=model
          api_key = "EMPTY"
          api_base = "http://localhost:8000/v1"

      client = OpenAI(api_key=api_key, base_url=api_base)
      print("Model:", setmodel)
      stream = client.chat.completions.create(
          model=setmodel,
          messages=prompt,
          stream=True,
          max_tokens=token,
          temperature=temp,
          extra_body={
          "stop_token_ids": [7]
          }
      )
      final_response = ""
      for chunk in stream:
          if chunk.choices[0].delta.content is not None:
              if streamprint:
                  print(chunk.choices[0].delta.content, end="")
              final_response += chunk.choices[0].delta.content  
      return final_response                
  text=text[0:4096]
  input=[
    {
      "role": "user",
      "content": '''Please execute the following steps to extract all proper nouns from the input text:

Step 1: Identification

    Identify all proper nouns that are specific to the field of computer science.

Inclusion Criteria (Rule1):

    Only include proper nouns that are must directly mentioned in the text.
    Focus on specific names related to:
        Malware
        Unique vulnerabilities (specifically referenced by CVE identifiers)
        Distinct network identifiers (IP addresses, domain names, URLs)
        Detailed Indicators of Compromise (IOCs) such as file paths, hash values, port numbers, registry key changes, MAC addresses, and unique signatures.
        Software names (e.g., antivirus programs, hacking tools, operating systems)
        Hardware names (e.g., specific models of routers, servers, or IoT devices)
        Specific names of hacker groups or threat actors
        Specific names of security protocols or encryption algorithms
        Specific names of programming languages, libraries, or frameworks
        Specific names of computer science concepts or technologies
        
Step 2: Exclusion
    Remove all:
        Human names
        Country names
        City names

Step 3: Output Formatting

    Provide the output in the format of a Python list: ["name1", "name2", ...]. 
    Make sure every name should be enclosed in double quotes and separated by a comma!
    After listing all names according to Rule1, evaluate each against Rule2.
    For each name you exclude based on Rule2, provide a concise justification. If you don't exclude any names, you still start output with "Final list based on Rule2: []".

Final Step: Final Output

    After the evaluation, present the final, curated list that adheres to Rule1 and Rule2.'''
      
    },
    {
      "role": "assistant",
      "content": "Understood. I will now proceed to extract the names."
    },
    {
      "role": "user",
      "content": "ExampleMalware1 is a malware that is designed to run as a deleter in Country1. ExampleMalware1 sample is designed to run as one-time encryptor. It is an update version of ExampleMalware2 from human1."
    },
    {
      "role": "assistant",
      "content": "[\"ExampleMalware1\", \"ExampleMalware2\", \"Country1\", \"human1\"]. Then I list every name and think if each name should be excluded based on Rule2.\n ExampleMalware1: Include.\n ExampleMalware2: Include. Country1: Exclude,country name.\n.\n human1: Exclude, human name.\nFinal list based on Rule2: [\"ExampleMalware1\", \"ExampleMalware2\"]."
    },
    {
      "role": "user",
      "content": "Good job! Now, I will give you another new sentence. Here is the sentence: \""+text+"\""}
  ]
  import ast,json
  max_retries = 2
  for attempt in range(max_retries):
    try:
      answer = ask(input, 4096, 1, "./cti-qwen1.5-70b-awq", False)
      if 'Rule2' in answer:
        list_str_answer = answer[answer.rindex('Rule2'):]
      else:
        list_str_answer = answer
      list_str_answer = list_str_answer[list_str_answer.rindex('['):list_str_answer.rindex(']')+1]
      print('\nName entity recognition result raw:', answer)
      if '\'' not in list_str_answer and '\"' not in list_str_answer:
          stripped_string = list_str_answer.strip('[]')
          elements = stripped_string.split(', ')
          quoted_elements = ["'{}'".format(element) for element in elements]
          list_str_answer = '[' + ', '.join(quoted_elements) + ']'
      print('\nName entity recognition result with only list:', list_str_answer)
      answer = ast.literal_eval(str(list_str_answer))
      return answer

    except Exception as e: 
      print(f"Attempt {attempt+1} failed with error: {e}")
      if attempt == max_retries - 1:  
          return ['BlankWordHere']

    return ['BlankWordHere']


import os
import random
import string
import pandas as pd  # Assuming longmem_cache is a pandas DataFrame

def save_to_unique_filename(article_raw_fulltext, longmem_cache):
    def generate_unique_filename_from_string(input_string):
        # Trim spaces from the ends of the string
        trimmed_string = input_string.strip()

        # Keep only the first 196 characters including only alphabet letters and spaces
        filtered_chars = [c for c in trimmed_string if c.isalpha() or c.isspace()]
        first_196_chars = ''.join(filtered_chars[:196])

        # Replace spaces with underscores
        base_filename = 'triples_' + first_196_chars.replace(' ', '_') + '.xlsx'

        # Check if the filename is unique
        if not os.path.exists(base_filename):
            return base_filename  # If it does not exist, return the original filename

        # Separate filename and extension
        file_name, file_extension = os.path.splitext(base_filename)

        # Attempt to create a unique filename by appending a counter
        for counter in range(2, 1000):  # Try from 2 to 999
            new_filename = f"{file_name}_step{counter}{file_extension}"
            if not os.path.exists(new_filename):
                return new_filename  # Return new filename if it does not exist

        # If a unique filename is not found, use a random letter
        random_letter = random.choice(string.ascii_letters)
        new_filename = f"{file_name}_step{random_letter}{file_extension}"
        return new_filename

    # Generate a unique filename based on the article text
    unique_filename = generate_unique_filename_from_string(article_raw_fulltext)

    # Try to save the DataFrame to an Excel file
    try:
        longmem_cache.to_excel(unique_filename, index=False)
        print(f"Save to: {unique_filename}")
    except Exception as e:
        print(f"Save fail with: {e}")


def full_article_to_longmem(single_article,enhance=False,realtime_othernlp=True):
    resolved_text=single_article 
    import concurrent.futures
    from collections import Counter
    def run_parallel_entity_recognition(text):
        results = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
            futures = [executor.submit(name_entity_recognition, text) for _ in range(3)]
            for future in concurrent.futures.as_completed(futures):
                results.append(set(future.result()))
        
        for i, result in enumerate(results):
            print(f'Results {i+1}:', result)
        
        entity_counts = Counter()
        for result in results:
            entity_counts.update(result)
        
        common_entities = {entity for entity, count in entity_counts.items() if count >= 2}
        
        discard_entities = {entity for entity, count in entity_counts.items() if count < 2}
        print('Common Entities:', common_entities)
        print('Discard Entities:', discard_entities)

        return list(common_entities)

    keywords = run_parallel_entity_recognition(resolved_text)
    print('Keywords are:',keywords)
    #Get the dict_string_label here
    #if realtime_othernlp is True
    from nltk.tokenize import sent_tokenize
    sentences = sent_tokenize(resolved_text)
    dict_string_label={}
    import pickle
    from simpletransformers.classification import MultiLabelClassificationModel
    pkl_file = open('big_label_list.pkl', 'rb')
    big_label_list = pickle.load(pkl_file)
    pkl_file.close()
    if realtime_othernlp is True: 
        print('Realtime Simpletransformers')
        my_best_model_dir='./tactic model'
        model = MultiLabelClassificationModel("roberta", my_best_model_dir, num_labels=len(big_label_list))
        for sentence in sentences:
            predictions, raw_outputs = model.predict([sentence])
            predicted_labels = [big_label_list[i] for i, label in enumerate(predictions[0]) if label == 1]
            dict_string_label[sentence] = predicted_labels
    else:
        print('Load from offline sentence predictions')
        with open('offline_predictions_simple.pkl', 'rb') as f:
            dict_sentence_onehotlabel = pickle.load(f)
        for singlesentence in sentences:
            if singlesentence in dict_sentence_onehotlabel.keys():
                onehotlabel=dict_sentence_onehotlabel[singlesentence]
                predicted_labels=[big_label_list[i] for i, label in enumerate(onehotlabel) if label == 1]
                dict_string_label[singlesentence]=predicted_labels
            else:
                print('No prediction for this sentence:',singlesentence)
  
    grouped_texts_strings = full_text_to_parts(single_article)
    triple_cache = []
    text_cache = []
    for i in range(len(grouped_texts_strings)):
        if i >40:
            return
        this_time_test=grouped_texts_strings[i]
        if len(this_time_test) > 1500:
            this_time_test = this_time_test[0:1500]
        print('Working on：',this_time_test)
        this_keywords = []
        for single_keyword in keywords:
            if single_keyword.lower() in this_time_test.lower():
                this_keywords.append(single_keyword)
                
        highlight_sentence = []
        for key in dict_string_label.keys():
            if key.lower() in this_time_test.lower():
                highlight_sentence.append(key)
                
        if len(this_keywords) != 0 and enhance:
            print('Find keywords：',this_keywords)
        else:
            print('No keywords found, normal operation')
        if len(highlight_sentence) != 0 and enhance:
            print('Find CTI sentence label：',highlight_sentence)
        else:
            print('No CTI sentence label found, normal operation')
            
        triple = compute_full_extracted_triples(this_time_test, this_keywords, highlight_sentence,enhance)
            
        print('Get triples：',triple)
        clean_triple_forMEM = clean_full_extracted_triples(triple)
        if  'Formbook' in clean_triple_forMEM or 'XLoader' in clean_triple_forMEM or 'savetextfile' in clean_triple_forMEM or 'Leafminer' in clean_triple_forMEM or 'FinSpy' in clean_triple_forMEM or 'Kismet' in clean_triple_forMEM or 'Agumon' in clean_triple_forMEM or 'Gabumon' in clean_triple_forMEM or 'Biyomon' in clean_triple_forMEM or '2042' in clean_triple_forMEM or check_brackets(clean_triple_forMEM)==False:
            triple = compute_full_extracted_triples(this_time_test, this_keywords, highlight_sentence,enhance)
            clean_triple_forMEM = clean_full_extracted_triples(triple)
            
        if 'This text does not contain any computer science related triples'  in triple or 'Blank placeholders since the current text does not contain any computer science related triples' in triple:
            clean_triple_forMEM='This text does not contain any computer science related triples'
        else:
            if  'Formbook' in clean_triple_forMEM or 'XLoader' in clean_triple_forMEM or 'savetextfile' in clean_triple_forMEM or 'Leafminer' in clean_triple_forMEM or 'FinSpy' in clean_triple_forMEM or 'Kismet' in clean_triple_forMEM or 'Agumon' in clean_triple_forMEM or 'Gabumon' in clean_triple_forMEM or 'Biyomon' in clean_triple_forMEM or '2042' in clean_triple_forMEM or check_brackets(clean_triple_forMEM)==False:
                triple = compute_full_extracted_triples(this_time_test)
                clean_triple_forMEM = clean_full_extracted_triples(triple)
            
            if  'Formbook' in clean_triple_forMEM or 'XLoader' in clean_triple_forMEM or 'savetextfile' in clean_triple_forMEM or 'Leafminer' in clean_triple_forMEM or 'FinSpy' in clean_triple_forMEM or 'Kismet' in clean_triple_forMEM or 'Agumon' in clean_triple_forMEM or 'Gabumon' in clean_triple_forMEM or 'Biyomon' in clean_triple_forMEM or '2042' in clean_triple_forMEM or check_brackets(clean_triple_forMEM)==False:
                triple = compute_full_extracted_triples(this_time_test)
                clean_triple_forMEM = triple
            
        print('Get short-term memory :')
        print(clean_triple_forMEM)
        
        if i == 0:
            if check_brackets(clean_triple_forMEM):
                longmem = clean_triple_forMEM
            else:
                longmem = 'No longterm memory'
            triple_cache.append(clean_triple_forMEM)
            text_cache.append(this_time_test)
            print('Finish the first paragraph')
            import pandas as pd
            try:
                new_data = pd.DataFrame({'single_article': [str(single_article)], 'longmem': [str(longmem)],'totalkeywords':[str(keywords)],'totalhighlightsentence':[str(dict_string_label)],'thiskeywords':[str(this_keywords)],'thishighlightsentence':[str(highlight_sentence)]})
            except:
                new_data = pd.DataFrame({'single_article': [str(single_article)], 'longmem': [str(longmem)]})
            save_to_unique_filename(single_article, new_data)
        if i> 40:
            print('Too many paragraphs, stop here')
        if i >= 1 and i<=40:
            print('Start to merge the short-term memory to long-term memory')
            print(longmem)
            original_longmem=longmem
            if len(longmem)>=1000:
                    longmem=longmem[-1000:]
                    if '[' in longmem:
                        longmem=longmem[longmem.index('['):]
            if check_brackets(clean_triple_forMEM):
                max_retries = 3  # 最大重试次数
                retry_count = 0  # 重试计数器
                while retry_count < max_retries:
                    newlongmem = merge_extracted_triples(longmem, clean_triple_forMEM, this_time_test)
                    print('newlongmem is:')
                    print(newlongmem)
                    newlongmem=newlongmem.replace('-The start of the new short-term memory area-','-The start of new short-term memory area-')
                    newlongmem=newlongmem.replace('-The end of the new short-term memory area-','-The end of new short-term memory area-') 
                    if '-The start of new short-term memory area-' in newlongmem and '-The end of new short-term memory area-' in newlongmem:
                        newlongmem=newlongmem[newlongmem.rindex('-The start of new short-term memory area-')+len('-The start of new short-term memory area-'):newlongmem.rindex('-The end of new short-term memory area-')]
                        if not any(keyword in newlongmem for keyword in ['Formbook', 'XLoader', 'savetextfile', 'Leafminer', 'FinSpy', 'Kismet','Agumon','Gabumon','Biyomon','2042']):
                            longmem = str(original_longmem)+', '+str(newlongmem)
                            retry_count=9999
                        else:
                            retry_count += 1
                    else:
                        retry_count += 1
            else:
                longmem=original_longmem
            print('get new longmem')
            print(longmem)      
            import pandas as pd
            try:
                new_data = pd.DataFrame({'single_article': [str(single_article)], 'longmem': [str(longmem)],'totalkeywords':[str(keywords)],'totalhighlightsentence':[str(dict_string_label)],'thiskeywords':[str(this_keywords)],'thishighlightsentence':[str(highlight_sentence)]})
            except:
                new_data = pd.DataFrame({'single_article': [str(single_article)], 'longmem': [str(longmem)]})
                
            longmem_cache = new_data
            
            
            import os
            import random
            import string
            
            def generate_unique_filename_from_string(input_string):
                trimmed_string = input_string.strip()

                filtered_chars = [c for c in trimmed_string if c.isalpha() or c.isspace()]
                first_64_chars = ''.join(filtered_chars[:196])

                base_filename = 'triples_'+first_64_chars.replace(' ', '_') + '.xlsx'
                if not os.path.exists(base_filename):
                    return base_filename  

                file_name, file_extension = os.path.splitext(base_filename)
                
                for counter in range(2, 1000):  
                    new_filename = f"{file_name}_step{counter}{file_extension}"
                    if not os.path.exists(new_filename):
                        return new_filename 
                
                random_letter = random.choice(string.ascii_letters)
                new_filename = f"{file_name}_step{random_letter}{file_extension}"
                return new_filename
            
            full_article_filename = generate_unique_filename_from_string(input_string=single_article)
            
            full_path = full_article_filename

            try:
                longmem_cache.to_excel(full_path, index=False)
                print(f"Save to：{full_path}")
            except Exception as e:
                print(f"Get error：{e}")
                
    return longmem

Read data from 

In [ ]:
#Loading data
import pandas as pd
target= pd.read_csv('target.csv')
targettext=target.loc[0,'string']
input_articles=target['string'].tolist()

Start extracting from articles

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import pickle
def process_article(article, index):
    try:
        full_article_to_longmem(article,enhance=True,realtime_othernlp=True)
        with open("done.txt", "a") as file:
            file.write(article + "\n")
        with open("done_index.txt", "a") as file:
            file.write(str(index))
    except Exception as e:
        with open("error.txt", "a") as file:
            file.write(article + "\n")
        print(f"Failed on：{e}")


pool = ThreadPoolExecutor(max_workers=8)
for i, article in enumerate(input_articles):
    try:
        pool.submit(process_article, article, i)
    except Exception as e:
        print(f"Failed on：{e}")
        continue

pool.shutdown()